In [1]:
"""
1) Fixed flux two-fluid model (VKH linear stability analysis)
"""
# First attempt to simplified ffm-tfm (it works)
# Libraries
from fenics import *
from IPython.display import clear_output
from scipy.optimize import brenth
from scipy.linalg import eig, eigh
from scipy.sparse.linalg import eigs, eigsh
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from matplotlib import rc
from scipy.sparse import csr_matrix
from numpy import linalg as LA
from scipy import sparse

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import ufl
%matplotlib inline

H        = 0.078
alpha1   = 0.5

# stable
# j1       = 0.05
# j2       = 0.829

# # unstable
# j1       = 0.5
# j2       = 6.908

#ill posed
j1       = 0.7
j2       = 10.0

j        = j1 + j2

u1       = j1/alpha1
u2       = j2/(1 - alpha1)

rho1     = 1000
rho2     = 1.1614
r        = rho2/rho1

g        = 9.8
thetadeg = 0
theta    = pi*thetadeg/180
L        = 1                      # m, pipe length

# Form compiler options
parameters ['form_compiler']['representation'] = 'uflacs'
parameters ["form_compiler"]["optimize"]          = True
parameters ["form_compiler"]["cpp_optimize"]      = True

# parameters ["form_compiler"]['precision'] = 50
parameters["form_compiler"]["cpp_optimize_flags"] = "-O3 -ffast-math -march=native"
# parameters ["form_compiler"]["quadrature_degree"] = 8

parameters["allow_extrapolation"] = True
parameters["refinement_algorithm"] = "plaza_with_parent_facets"

# Define constants
var1bc1     = Constant(alpha1)                    # alpha1           1 - u1_0
var2bc1     = Constant(u1)                        # u1               u3_0
L           = Constant(L)

# Periodic boundary condition
class PeriodicBoundary(SubDomain):
    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
          return bool(x[0] < DOLFIN_EPS and x[0] > - DOLFIN_EPS and on_boundary)
    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
          y[0] = x[0] - 1.0     
            
# Define space discretization properties
xmin        = 0
xmax        = L
nx          = 40                            # no. of elements 100 to 1600
p           = 1                              # degree of FEM
mesh        = IntervalMesh (nx, xmin, xmax)

# Define function space
V1          = FiniteElement ('Lagrange', mesh.ufl_cell(), degree = p)
V2          = FiniteElement ('Lagrange', mesh.ufl_cell(), degree = p+1)
element     = MixedElement ([V1, V2])

# V           = FunctionSpace(mesh, element)
V          = FunctionSpace(mesh, element, constrained_domain = PeriodicBoundary())
# ?(1)

# Define test and trial functions
v     = TestFunction (V)
v1, v2      =  split (v)
var         = TrialFunction (V)
var1, var2  = split (var)

# Dirichlet boundary condition
def left (x, on_boundary): 
    return x[0] < DOLFIN_EPS

bc1         = DirichletBC(V.sub (0), var1bc1, left)
bc2         = DirichletBC(V.sub (1), var2bc1, left)
bcs         = [bc1, bc2]








# Linear stability

# Define constants, expressions and functions used in weak form
j       = Constant (j)
r       = Constant (r)
g       = Constant (g)
H       = Constant (H)
theta   = Constant (theta)
#k       = Constant (dt)

f1      = 0.005
f2      = 0.000
fi      = 0.014

f1      = Constant (f1)
f2      = Constant (f2)
fi      = Constant (fi)

vel2_bc1 = (j - var1bc1*var2bc1)/(1 - var1bc1)

B11_bc1  = var2bc1
B12_bc1  = var1bc1
B21_bc1  = var2bc1
B22_bc1  = r*(vel2_bc1 - var2bc1)**2/(1 - var1bc1) - g*cos(theta)*H

F21_bc1  = - ((f1*var2bc1)/(H*var1bc1)) - (f2*r*var1bc1*(j - var1bc1*var2bc1))\
    /(H*(1 - var1bc1)**3) + fi*r*(1/(H*(1 - var1bc1)) + 1/(H*var1bc1))*\
    ( - 1 - var1bc1/(1 - var1bc1))*( - var2bc1 + (j - var1bc1*var2bc1)/(1 - var1bc1))

F22_bc1  = (j*r*(fi + f2*var1bc1**2) - (f1*( - 1 + var1bc1)**3 + r*(fi + f2*var1bc1**3)) \
    *var2bc1)/(H*( - 1 + var1bc1)**3*var1bc1)

B11_bc1      = Constant (B11_bc1)
B12_bc1      = Constant (B12_bc1)
B21_bc1      = Constant (B21_bc1)
B22_bc1      = Constant (B22_bc1)
F21_bc1      = Constant (F21_bc1)
F22_bc1      = Constant (F22_bc1)







# Define weak form (semi distretised stifness matrix with artificial difussion)
a  = ( inner(B11_bc1*Dx(var1,0), v1)\
     + inner(B12_bc1*Dx(var2,0), v1)\
     + inner(B21_bc1*Dx(var2,0), v2)\
     - inner(B22_bc1*Dx(var1,0), v2))*dx

b  = -(inner(F21_bc1*var1, v2)\
     + inner(F22_bc1*var2, v2))*dx

# dummy  = (  inner(Constant(0),v1)\
#       + inner(Constant(0),v2)  )*dx

# dummy = v[0]*dx

# Assemble stiffness form
A = PETScMatrix()
B = PETScMatrix()

# assemble_system(F1, L, bcs, A_tensor = A, b_tensor = b)
# assemble_system(a, dummy, bcs, A_tensor=A)
# assemble_system(b, dummy, bcs, A_tensor=B)

assemble(a - b, tensor = A)
# assemble(b, tensor=B)

# [bc.apply(B) for bc in bcs]
# [bc.zero(B) for bc in bcs]
# [bc.apply(A) for bc in bcs]

A_mat = as_backend_type(A).mat()
# B_mat = as_backend_type(B).mat()

# Transform to numpy array
A_sparray = csr_matrix(A_mat.getValuesCSR()[::-1], shape = A_mat.size)
# B_sparray = csr_matrix(B_mat.getValuesCSR()[::-1], shape = B_mat.size)

# Print array
# print(A_sparray)

# Plot array
plt.figure (3, figsize = (12, 6))
plt.spy(A_sparray)
plt.grid (True, which = "both")
# plt.rcParams ['figure.figsize'] = [12, 8]

plt.show



ModuleNotFoundError: No module named 'fenics'

In [ ]:
solver = SLEPcEigenSolver (A)
# print("Computing eigenvalues. This can take a minute.")

#https://fenicsproject.org/docs/dolfin/1.4.0/python/programmers-reference/cpp/la/SLEPcEigenSolver.html
solver.parameters["solver"]             = "arnoldi"
solver.parameters["problem_type"]       = "non_hermitian"
solver.parameters["spectrum"]           = "target magnitude"
# solver.parameters["spectral_transform"] = "shift-and-invert"
# solver.parameters["spectral_shift"]     = 0.5

# solver.parameters['solver'] = "arnoldi"
# solver.parameters["problem_type"]       = "non_hermitian"

solver.solve ()

listreal = []
listimag = []

for ii in range(solver.get_number_converged()):
    r, c, rx, cx = solver.get_eigenpair(ii)
#     print ("i = ", ii)
    # print ("real      =", r)
    listreal.append (r)

    # print ("imaginary =", c)
    listimag.append (c)

# Extract largest (first) eigenpair
r, c, rx, cx = solver.get_eigenpair(0)

# print("Largest eigenvalue: ", r)
            
# Characteristics for plots
liststyles       = ["--", "-", "-.", "."]
listcolor        = ["k", "g", "b", "r", "none"]
listmarkers      = ["s", "o", "^", ">", "<", "p"]

# Plot eigenspectra 1
fig, ax = plt.subplots ()
area = 50

ax.scatter (listreal, listimag, s = area, marker = listmarkers [0], color = listcolor [4], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5)
# ax.set_xscale ('symlog', linthreshx = 1e-4)

plt.rcParams ['figure.figsize'] = [12, 8]
# leg1 = ax.legend (loc = 'upper right', frameon = True, fontsize = 14);
plt.grid (True, which = "both")
plt.xlabel ('Re $[\omega]$ [1/s]', fontsize = 18)
plt.ylabel ('Im $[\omega]$ [1/s]', fontsize = 16)

matplotlib.rc ('xtick', labelsize = 18)     
matplotlib.rc ('ytick', labelsize = 18)

# plt.ylim (( - 0.002 , ))
# plt.xlim (( 2*vals.real[localminreal[0]], 2*vals.real[localmaxreal[0]]))
# plt.show ()    

# sorting the list 
listreal.sort() 

# printing the last element 
print("Largest element is:", listreal[-1]) 
